In [115]:
import requests
import json
import urllib
import pandas as pd
from joblib import Parallel, delayed
from progressbar import ProgressBar as Progress
import os
import sys
import hashlib
import urllib.request
import os, fnmatch, gzip, shutil, tarfile
from pathlib import Path

In [116]:
# Program to return gene expression data from GDC
Location = '/Users/colinreid/Desktop/Brain RNASeq'

# Read in manifest file
Manifest_Loc = '/Users/colinreid/Downloads/gdc_manifest.2018-01-26T20_35_54.714476.txt'
Manifest = pd.read_csv(Manifest_Loc,delimiter='\t')
Manifest

,id,filename,md5,size,state
0,f668fc27-d2f5-488b-904e-ec6de98fb813,02d43414-c8c9-42d3-8009-f9fb164dc8e0.htseq.cou...,6e0590a5208efaf22879032cecf10b88,253886,live
1,07b21ab7-9c14-4bf2-b692-3449e7c8daf0,b004e38d-f48c-49f9-a027-4a702fbaf107.htseq.cou...,ddf94dcd2b37abefc36b5d42753bc3bd,245536,live
2,7f58b7ed-10c8-4b45-92ba-61aaa9904fe7,fe179a60-5e81-48c1-b437-7c38ed910ba8.htseq.cou...,f44556681df906eb9e3147990ba6fc64,255542,live
3,56e47486-4d33-4fb1-9653-5a4043216e86,e87cfbaa-791e-4a6b-8ec8-28df636e7cdc.FPKM.txt.gz,d016f1a9e1c48284949b5dc5c44ac249,525791,live
4,3481cb59-e025-4bbb-80a8-11285a5bd8aa,19f1375f-51f2-4f4c-88fe-6fc84c37e5b6.FPKM-UQ.t...,3b0288dfdc71ebd88d878f0277540061,533043,live
5,628792e2-4e0c-4b49-b6b1-af80d2b6dccf,21800024-cf76-4185-b57b-526539ccdba2.htseq.cou...,0c7ea2f1897a08aaca06071072b1577b,254920,live
6,873a4e74-cfb3-46ff-939d-22afa727fd6c,44ed61ae-e74e-40d7-98ce-7803e040567f.FPKM.txt.gz,82a43da3ebaef4e3bf29b00f76aa2d29,543668,live
7,70732ce6-2ce7-4ea0-92f0-8a15032b64dc,746ed118-eb9f-43a5-bcda-872999ae7128.FPKM.txt.gz,9ab95c5a0ab5c2d048e294d024a19657,539598,live
8,5278a216-f2e3-444b-9ac8-31c8c46c88f4,502fc9a8-6409-4b7a-a0f8-abed308ce84a.htseq.cou...,5886b84cb73c1e9abfc07c8404e3969c,255698,live
9,69595b5c-5d7b-416b-aa98-b2218bbe5d7b,76c6e110-9a94-4fcf-a534-a21ba4698f86.htseq.cou...,1233a2667bbfb3fffee7df16b4186925,254454,live


In [117]:
# JSON Filters Constructor
class Filter(object):
    
    def __init__(self):
        self.filter = {"op": "and","content": []}
        
    def add_filter(self, Field, Value, Operator):
        self.filter['content'].append({"op":Operator,"content":{"field":Field,"value":Value}})
        
    def create_filter(self):
        self.final_filter = json.dumps(self.filter,separators=(',',':'))
        return self.final_filter

    def reset_filter(self):
        self.filter = {"op": "and","content": []}

# Convert JSON to Percent Encoded String
def JSONtoPctEncodeStr(JSON):
    PctEncodedStr = urllib.parse.quote(JSON)
    return PctEncodedStr

# Create HTTP Request given Inputs
def GetRequest(API_Flag,EndPoint,Filters,Fields,Size): # Create API Requests and append them to a list titled 'Requests'
    if API_Flag == 'legacy':
        API_URL = 'https://api.gdc.cancer.gov/legacy/'
    else:
        API_URL = 'https://api.gdc.cancer.gov/'
        
    Request = API_URL + EndPoint + '?' + 'filters=' + Filters + '&' + 'fields=' + Fields + '&' + 'size=' + Size 
    
    return Request

In [131]:
UUIDs = list(Manifest['id'])

# counts
File_Filter.add_filter("files.file_id",UUIDs,"in")
File_Filter.add_filter("files.analysis.workflow_type",["HTSeq - Counts","HTSeq - FPKM","HTSeq - FPKM-UQ"],"in")
File_Filter.create_filter()

EndPoint = 'files'
Fields = 'cases.samples.portions.analytes.aliquots.submitter_id,file_name,cases.samples.sample_type,file_id,md5sum,experimental_strategy,analysis.workflow_type'
Size = '10000'

Payload = {'filters':File_Filter.create_filter(),
           'format':'json',
           'fields':Fields,
           'size':Size}
r = requests.post('https://api.gdc.cancer.gov/files', json=Payload)
data = json.loads(r.text)
file_list = data['data']['hits']

In [132]:
Data_Dict = []
for file in file_list:
    UUID = file['file_id']
    File_Name = file['file_name']
    Barcode = file['cases'][0]['samples'][0]['portions'][0]['analytes'][0]['aliquots'][0]['submitter_id']
    MD5 = file['md5sum']
    Sample_Type = file['cases'][0]['samples'][0]['sample_type']
    Experimental_Strategy = file['experimental_strategy']
    Workflow_Type = file['analysis']['workflow_type']
    
    Data_Dict.append({'UUID':UUID,'File Name':File_Name,'TCGA Barcode':Barcode,'MD5':MD5,'Sample Type':Sample_Type,'Experimental Strategy':Experimental_Strategy,'Workflow Type':Workflow_Type})

In [133]:
Data_Frame = pd.DataFrame(Data_Dict)
Data_Frame

,Experimental Strategy,File Name,MD5,Sample Type,TCGA Barcode,UUID,Workflow Type
0,RNA-Seq,02d43414-c8c9-42d3-8009-f9fb164dc8e0.htseq.cou...,6e0590a5208efaf22879032cecf10b88,Primary Tumor,TCGA-15-0742-01A-01R-1850-01,f668fc27-d2f5-488b-904e-ec6de98fb813,HTSeq - Counts
1,RNA-Seq,b004e38d-f48c-49f9-a027-4a702fbaf107.htseq.cou...,ddf94dcd2b37abefc36b5d42753bc3bd,Primary Tumor,TCGA-32-4213-01A-01R-1850-01,07b21ab7-9c14-4bf2-b692-3449e7c8daf0,HTSeq - Counts
2,RNA-Seq,fe179a60-5e81-48c1-b437-7c38ed910ba8.htseq.cou...,f44556681df906eb9e3147990ba6fc64,Recurrent Tumor,TCGA-06-0210-02A-01R-2005-01,7f58b7ed-10c8-4b45-92ba-61aaa9904fe7,HTSeq - Counts
3,RNA-Seq,e87cfbaa-791e-4a6b-8ec8-28df636e7cdc.FPKM.txt.gz,d016f1a9e1c48284949b5dc5c44ac249,Primary Tumor,TCGA-16-1045-01B-01R-1850-01,56e47486-4d33-4fb1-9653-5a4043216e86,HTSeq - FPKM
4,RNA-Seq,19f1375f-51f2-4f4c-88fe-6fc84c37e5b6.FPKM-UQ.t...,3b0288dfdc71ebd88d878f0277540061,Primary Tumor,TCGA-02-2483-01A-01R-1849-01,3481cb59-e025-4bbb-80a8-11285a5bd8aa,HTSeq - FPKM-UQ
5,RNA-Seq,21800024-cf76-4185-b57b-526539ccdba2.htseq.cou...,0c7ea2f1897a08aaca06071072b1577b,Primary Tumor,TCGA-27-2519-01A-01R-1850-01,628792e2-4e0c-4b49-b6b1-af80d2b6dccf,HTSeq - Counts
6,RNA-Seq,44ed61ae-e74e-40d7-98ce-7803e040567f.FPKM.txt.gz,82a43da3ebaef4e3bf29b00f76aa2d29,Primary Tumor,TCGA-06-0646-01A-01R-1849-01,873a4e74-cfb3-46ff-939d-22afa727fd6c,HTSeq - FPKM
7,RNA-Seq,746ed118-eb9f-43a5-bcda-872999ae7128.FPKM.txt.gz,9ab95c5a0ab5c2d048e294d024a19657,Primary Tumor,TCGA-06-5856-01A-01R-1849-01,70732ce6-2ce7-4ea0-92f0-8a15032b64dc,HTSeq - FPKM
8,RNA-Seq,502fc9a8-6409-4b7a-a0f8-abed308ce84a.htseq.cou...,5886b84cb73c1e9abfc07c8404e3969c,Primary Tumor,TCGA-76-4929-01A-01R-1850-01,5278a216-f2e3-444b-9ac8-31c8c46c88f4,HTSeq - Counts
9,RNA-Seq,76c6e110-9a94-4fcf-a534-a21ba4698f86.htseq.cou...,1233a2667bbfb3fffee7df16b4186925,Primary Tumor,TCGA-06-0138-01A-02R-1849-01,69595b5c-5d7b-416b-aa98-b2218bbe5d7b,HTSeq - Counts


In [134]:
# Location to save files as they are downloaded
OFILE = {'data':Location+"{ES}/{WF}/{uuid}/{name}"}

In [135]:
PARAM = {
# Number of parallel HTTP requests
'requests' : 10,

# URLS
'url-data' : "https://api.gdc.cancer.gov/data/{uuid}",
'url-info' : "https://api.gdc.cancer.gov/files/{uuid}",

# Persistence upon error
'max retry' : 10,
}

In [136]:
# Function for downloading files
def job(uuid, name, md5, ES, WF, retry=0):
    if retry : sleep(retry)
    try :
            fout = OFILE['data'].format(ES=ES, WF=WF, uuid=uuid, name=name)
            def md5_ok() :
                with open(fout, 'rb') as f :
                    return (md5 == hashlib.md5(f.read()).hexdigest())

            do_download = not (os.path.isfile(fout) and md5_ok())

            if do_download :
                print("Downloading (attempt {}): {}".format(retry, uuid))
                url = PARAM['url-data'].format(uuid=uuid)

                with urllib.request.urlopen(url) as response :
                    data = response.read()

                os.makedirs(os.path.dirname(fout), exist_ok=True)

                with open(fout, 'wb') as f :
                    f.write(data)
            return (uuid, retry, md5_ok())
    except Exception as e :
        print("Error (attempt {}): {}".format(retry, e))        
        if (retry >= PARAM['max retry']) :
            raise e
        return job(uuid, name, md5, retry + 1)

In [137]:
# Launch download jobs
OK = Parallel(n_jobs=PARAM['requests'], batch_size=5, verbose=0)(
delayed(job)(*q)
for q in Progress()(sorted(zip(Data_Frame['UUID'],Data_Frame['File Name'],Data_Frame['MD5'],Data_Frame['Experimental Strategy'],Data_Frame['Workflow Type'])))
)

100% |########################################################################|


In [ ]:
# Function that unpacks gz files into another directory
def gunzip(file_path,output_path):
    with gzip.open(file_path,"rb") as f_in, open(output_path,"wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Function to merge the files
p = Path(Location) # Create path object from the directory

ESs = list(Data_Frame['Workflow Type'].unique())

GZipLocs = [Location + 'RNA-Seq/' + ES for ES in ESs]

for i in range(len(ESs)):
    
    print('--------------')
    # Find all .gz files and ungzip into the folder
    pattern = '*.gz'
    Files = []

    # Create .gz directory in subfolder
    if not os.path.exists(GZipLocs[i] + '/UnzippedFiles/'):
        os.makedirs(GZipLocs[i] + '/UnzippedFiles/')

    for root, dirs, files in os.walk(GZipLocs[i]):
        for filename in fnmatch.filter(files, pattern):
            OldFilePath = os.path.join(root, filename)
            NewFilePath = os.path.join(GZipLocs[i] + '/UnzippedFiles/', filename.replace(".gz",".tsv"))

            gunzip(OldFilePath, NewFilePath) # unzip to New file path

            Files.append(NewFilePath) # append file to list of files

    # Create matrices containing pandas data frames of RNASeq file contents
    # Loop through all files and add contents as a pandas data frame to the matrix
    Matrix = {}

    print('Reading ' + ESs[i] + ' Files...')

    for file in Files:
        p = Path(file)
        Name = str(p.name).replace('.tsv','')
        Name = Data_Frame[Data_Frame['File Name'] == Name + '.gz']['TCGA Barcode'].to_string(index=False)
        Counts_DataFrame = pd.read_csv(file,sep='\t',header=None,names=['GeneId', Name])
        Matrix[Name] = tuple(Counts_DataFrame[Name])

    print('Merging files...')

    # Merge Matrices to dataframes and write to files
    if len(Matrix) > 0:
        print('Creating merged ' + ESs[i] + ' File...')
        Counts_Final_Df = pd.DataFrame(Matrix, index=tuple((Counts_DataFrame['GeneId'])))
        Counts_Final_Df.to_csv(str(Location) + '/Merged_'+ ESs[i] + '.tsv',sep='\t',index=True)

--------------
Reading HTSeq - Counts Files...
f6a99d72-7ef2-4c9c-880c-a26bd5fef9c7.htseq.counts
8a24d983-1fc9-4f5d-8fe8-aa4ed2a21f7a.htseq.counts
c341aa36-a431-477a-9fae-4550c4eea047.htseq.counts
6520f1eb-78e5-4459-b25b-62b5d6696bc0.htseq.counts
661fe6f6-26a0-457e-9a24-ae56f0ec9106.htseq.counts
ccc4ba5b-15f6-40f2-a3b8-187b38c4e6df.htseq.counts
dfa2ddfb-c161-4c8c-a538-be6e00a8466e.htseq.counts
57680511-b7a1-48d8-a79f-32d0a340c501.htseq.counts
167b3fae-0a99-4202-94ef-c83648f54fa9.htseq.counts
000f9b51-4747-4d7f-9ecc-25bef9b25ad7.htseq.counts
1d607fb7-679e-41cb-afa3-0faec33a1a81.htseq.counts
8e75a58b-642a-45be-bae1-0a0f7b75996d.htseq.counts
1be9778b-7fa2-4807-b6f4-2eebda72b2e3.htseq.counts
8a496ad8-98b2-4ff8-a21d-81db99fbb65b.htseq.counts
1098516b-cf0a-420e-86f0-bba5d868a210.htseq.counts
7d0cbdee-8343-4fac-98ac-ef1b4e7e533b.htseq.counts
332ae23a-a860-438a-be1b-9dcb4e34bfed.htseq.counts
f1ca3671-4f35-426a-9137-f3a690278a24.htseq.counts
dd7c0da8-2df5-452b-8824-85858b8f5dd7.htseq.counts
ab5

7306ff8a-6bf1-404c-8864-55d9c1bd2b97.htseq.counts
813fda52-3421-4489-b780-eed81f8bdca7.htseq.counts
a0a23823-3e04-49af-9494-87bc5ddd5424.htseq.counts
7dd9bc72-4979-4d27-af6f-638b226d37d5.htseq.counts
042bda6c-efe8-4341-a972-a0afe2e68a48.htseq.counts
36f0bd31-1f94-4063-8903-42d5bd113d0e.htseq.counts
19680925-a148-4f8a-af3b-9137746d663f.htseq.counts
1c3cb5b7-2c2c-4edc-a1d0-82e7fc19a63f.htseq.counts
1886f8fe-95fc-4774-91f9-fde0829dae5e.htseq.counts
a0a0da2d-8da4-4fe3-8d1c-2a5e2284d477.htseq.counts
a5882472-4653-44f2-aa1b-514ec25fa5db.htseq.counts
7f7121ff-1399-40da-a792-b7c31bc76570.htseq.counts
33958341-98b9-4bfa-89d3-fd087a7ef1e4.htseq.counts
0717d8b6-62aa-496e-9b0e-9ef841dec86e.htseq.counts
a5c6ef45-71ae-4011-9ee9-2193e6a5bc5f.htseq.counts
6fe364fb-d57e-4677-811c-d53535602669.htseq.counts
c3655184-869f-4099-9ce2-6d560914af7e.htseq.counts
63a30223-e2e9-45bc-bc42-c00f9202b493.htseq.counts
ba57f120-b348-431e-b7d7-8bae2196480e.htseq.counts
f5bb4817-1727-41f9-b7f6-3a4e1818c2df.htseq.counts


1d2a319d-c437-4479-9df8-dfbb040070b2.htseq.counts
41bff337-4561-4b22-829d-455a3fecc837.htseq.counts
ea5dce6f-9aeb-4cb4-b628-7377621563d5.htseq.counts
20aa9293-8cef-4469-9da1-d87969ec03b3.htseq.counts
f728fc58-a4b9-4525-971a-a74a7d676695.htseq.counts
d6cdbf9a-1009-4ee1-b041-a3744f4166ac.htseq.counts
272e4b8b-4174-4f29-9dd0-26c410898612.htseq.counts
456ef025-b7f0-4040-90dd-5bcc2e476bfc.htseq.counts
4612a611-2ff9-4f51-b4b2-af142254dbe8.htseq.counts
3c98fe32-4046-4c6f-a230-56bea0cd5bf9.htseq.counts
4b77b6fa-7626-43fb-8684-87e3c09276bb.htseq.counts
32bdde13-c526-426c-beb6-7d22b63b01c3.htseq.counts
9405ee49-6525-45e4-b471-04c6838c6514.htseq.counts
6508275a-e712-424e-bdaf-b9e1b07b0f95.htseq.counts
21badc40-5fb8-496d-a3e6-4f0704500a5d.htseq.counts
8b0dfb59-df19-4c93-b040-eb75bad863fc.htseq.counts
5ddf7c5c-a502-440a-826b-702ae27a21fe.htseq.counts
bcdc68db-c874-4097-9c46-b06e331caaf5.htseq.counts
53db73d4-b45c-46d9-99be-2f9588c017a0.htseq.counts
95b626ff-eb72-4539-b85b-2dea5a174fc2.htseq.counts


28ec98cf-4eb2-4da6-9752-2764142e8f72.htseq.counts
781aa454-9cb1-4421-90e6-0c57f7ee2830.htseq.counts
3a002b22-8465-4634-a35f-fe5c5940e43e.htseq.counts
21f38c14-7d7c-4156-8c85-88be68cae18d.htseq.counts
e85e20ef-dffc-47a0-9974-46f3adc8b10f.htseq.counts
e77be181-c686-4e02-80f6-dac52a80fb26.htseq.counts
3e859ca1-a1f6-4979-9a13-629a69373964.htseq.counts
d8e6b946-9f27-4f9a-9a0e-f9eacbf69bc8.htseq.counts
264a885b-95a5-456a-a821-140e65f6c008.htseq.counts
0084185c-3a20-4771-a036-5e80a30965ea.htseq.counts
a59a3ab2-2bc8-4745-b68c-15339de26484.htseq.counts
ffd727c7-0c28-4594-a5f1-689c62f2661d.htseq.counts
74a5a6c9-60ff-427d-9797-c4c8888aeeba.htseq.counts
c6045584-592e-412f-974a-c8ae2d4bb817.htseq.counts
4d4a1165-e304-4aed-9efa-7f6f5bacdb5e.htseq.counts
7824eae3-957c-4515-b128-45453a11624d.htseq.counts
6b015617-e989-4c20-b81f-15bf94d83a09.htseq.counts
e74e8b31-0938-4306-b4c5-b73d579f62e6.htseq.counts
22b2edfc-baf9-4a23-b87a-ae4c1b249664.htseq.counts
026a42e7-aae9-4d96-9e5a-e6159f94db11.htseq.counts


62727106-91c2-4f63-8ce5-d207569b38e4.htseq.counts
be2ad152-53a1-4566-8690-372652f2a565.htseq.counts
04d1f5f6-1dba-4daf-a641-dfe282ef4d82.htseq.counts
c34cfde5-3644-4763-a56a-9322678b9bb0.htseq.counts
84aa891c-c73e-48eb-bdbe-5774c2dc3c58.htseq.counts
3857e48b-89d6-46b8-b8d3-8b4814f95dab.htseq.counts
2c15cc20-2347-4bfc-826d-a062f6d74b46.htseq.counts
ec7f69e9-13b8-4880-ba63-4319810f1d4f.htseq.counts
f985e2f8-4375-4fda-a0c3-18030a146040.htseq.counts
06774da7-66ac-4a31-884a-7ea65cbcae93.htseq.counts
246ed833-5a5e-4c68-8ba9-cc42538ba5b7.htseq.counts
7a1742cf-010b-42e1-80d7-90151855dcbc.htseq.counts
657800a4-3bb0-4fef-8462-17a3565eb5b5.htseq.counts
046e804d-03d5-49c7-abcc-01af7f5a9208.htseq.counts
b25c8e4d-f6ca-4079-9120-b96147b9091e.htseq.counts
edfac2ec-e503-4e6a-aba7-33fcb0cb617f.htseq.counts
a308164e-b7e8-404f-bac2-72f586f25e78.htseq.counts
1e203b72-11b6-4188-95e6-c4989c40e824.htseq.counts
3f171964-37b4-43a2-b2f9-1b51dd924a7e.htseq.counts
02d43414-c8c9-42d3-8009-f9fb164dc8e0.htseq.counts


...

9b571cc1-447f-4fa1-b797-bf95c1ee2372.FPKM.txt
cbdf8c09-e3f4-4b3e-bdaa-f4a3a4e40068.FPKM.txt
40c5a89c-b4cb-49a8-b427-cea7f34b6cf7.FPKM.txt
b3850d71-3e90-4b9f-9057-55297d8b72bc.FPKM.txt
95aa2890-118d-48a8-a5d2-53caf27528b2.FPKM.txt
e1b718ca-2a1d-4270-b6d8-a833c42d7bab.FPKM.txt
78f9e89e-9447-4698-a79e-7acd0cb02927.FPKM.txt
1ad4c474-a8d9-481a-8298-b492c514245a.FPKM.txt
5cb8a505-e571-4faa-ab14-71e7b6a45a89.FPKM.txt
ccc4ba5b-15f6-40f2-a3b8-187b38c4e6df.FPKM.txt
57680511-b7a1-48d8-a79f-32d0a340c501.FPKM.txt
ab5f9e4e-ff6a-4353-83f4-5ffddca0050a.FPKM.txt
e2a56cc0-714e-4e7d-9c32-53e36e207c6b.FPKM.txt
18fbf794-a85b-4b0d-9a5c-c1c0e0ede3d8.FPKM.txt
5065f0e8-8a9b-4134-abcf-cbfad4928107.FPKM.txt
1b12501f-bd07-40ea-a763-1eca71f2123c.FPKM.txt
d55ac738-46b4-4e3e-b9e8-96969bfdda3a.FPKM.txt
e2a01ce0-a2fb-435a-9b05-7758e14d4157.FPKM.txt
e92dcfc6-d9d9-4525-910f-7a7c34478731.FPKM.txt
5af50081-3cae-41c5-8906-2c4004cc8bb5.FPKM.txt
57c3b272-7dcb-4937-83c1-02a3694e701f.FPKM.txt
3fe73640-0b2c-48f8-adde-00ebe9c749

d1809e31-9cd9-482b-8354-74df0a7841c0.FPKM-UQ.txt
e46e7af6-cbb2-4cb2-a1ec-47b68796cbd3.FPKM-UQ.txt
3857e48b-89d6-46b8-b8d3-8b4814f95dab.FPKM-UQ.txt
19f1375f-51f2-4f4c-88fe-6fc84c37e5b6.FPKM-UQ.txt
e9a0fc7b-94ea-477d-918a-1ef80fefefcd.FPKM-UQ.txt
0e30bd18-8e8b-4c52-aace-b5587c6df51a.FPKM-UQ.txt
b71157ef-fb33-4e47-87e3-ba68c673225c.FPKM-UQ.txt
5237d2f6-7bfe-4347-82d6-caebb8781ef9.FPKM-UQ.txt
e11893ad-2f46-4f9f-9fe3-56fe68349228.FPKM-UQ.txt
08fef39c-39be-41a8-80a9-be817d5ce44e.FPKM-UQ.txt
599629a4-f067-42f3-9a7f-8ee84f6e245b.FPKM-UQ.txt
dfb41de0-fd76-4884-aa76-5b0677bfeb7c.FPKM-UQ.txt
6e5b77f6-07d2-4bb3-8356-9a96bfa9b9e1.FPKM-UQ.txt
1a860864-2ba1-46ad-ad36-018a7b0ec08e.FPKM-UQ.txt
046e804d-03d5-49c7-abcc-01af7f5a9208.FPKM-UQ.txt
5a092f95-7774-41ed-b70c-58ed1daa0cf1.FPKM-UQ.txt
ceb8c083-f451-42f7-873d-8346caa0ffa3.FPKM-UQ.txt
d55ac738-46b4-4e3e-b9e8-96969bfdda3a.FPKM-UQ.txt
6cab3424-1d15-46ac-b858-e35df62e83dd.FPKM-UQ.txt
b80cedff-e07c-4325-975a-55d1d0bb5afc.FPKM-UQ.txt
a5c6ef45-71ae-4011-9

f8ac5011-0282-42e1-9f6a-57111ee49745.FPKM-UQ.txt
eb24e6f8-dc16-4a6d-b584-9046829cc2ef.FPKM-UQ.txt
2f7160fe-5f34-49bc-95fd-029ce83d8c42.FPKM-UQ.txt
93beb008-dd59-467d-86c4-c7a1e33313a3.FPKM-UQ.txt
7efa83ae-8a0c-4491-ac3b-606be61e16eb.FPKM-UQ.txt
982e2940-10b0-4383-bde8-f7ccb9920fc4.FPKM-UQ.txt
42959bfe-1a62-465f-aafc-684d973b43f7.FPKM-UQ.txt
a59a3ab2-2bc8-4745-b68c-15339de26484.FPKM-UQ.txt
09a93d5e-300f-4e7d-93f8-ed66c7bf8630.FPKM-UQ.txt
68fe871b-c631-469e-87a2-26130f416446.FPKM-UQ.txt
ddcfa6fc-4b82-4c8e-a53d-49b50cf3480e.FPKM-UQ.txt
3f558349-5e4d-41a2-95f1-4e2f6a2ac0d2.FPKM-UQ.txt
cf602f51-88b6-4251-9f5a-a27b321954fc.FPKM-UQ.txt
52771923-0324-47c6-a03a-d63d92834200.FPKM-UQ.txt
db5430ee-0016-4d9a-954c-a7845ab22ae2.FPKM-UQ.txt
2556ce90-db5f-4beb-897a-db3cf556e054.FPKM-UQ.txt
6da1dddf-9ad0-47b4-878c-c3ca359371dc.FPKM-UQ.txt
f67cf68d-56b3-451d-ba21-566664213691.FPKM-UQ.txt
93357150-89ae-4bd3-b9e7-bdc365e6d55e.FPKM-UQ.txt
e74e8b31-0938-4306-b4c5-b73d579f62e6.FPKM-UQ.txt
36f0bd31-1f94-4063-8

e6ff6ba9-8b49-4af5-80b4-efcc9cc3136b.FPKM-UQ.txt
a5525401-79be-4c7a-9107-71c022778d65.FPKM-UQ.txt
5065f0e8-8a9b-4134-abcf-cbfad4928107.FPKM-UQ.txt
8223f42b-434d-4966-a123-3b8a947884bc.FPKM-UQ.txt
cc6ba863-dbd3-497c-93ab-8b9cc9996c49.FPKM-UQ.txt
4d4a1165-e304-4aed-9efa-7f6f5bacdb5e.FPKM-UQ.txt
3d8aeff3-b4da-4bb4-bcdf-b7c15e88458c.FPKM-UQ.txt
9ae4ee25-88d4-4078-b6cc-f744c50dde8a.FPKM-UQ.txt
72003782-a8a0-422a-aed6-a103eb09994b.FPKM-UQ.txt
d92949ec-4287-42e2-ad63-9ef92dcae770.FPKM-UQ.txt
a4b7b2f6-d2f5-4213-a5a2-cabf2e2329da.FPKM-UQ.txt
8f6ec7f3-e417-4978-b866-ca4e488fada5.FPKM-UQ.txt
b5631f88-a4cd-4ec6-b0e9-30512aa38728.FPKM-UQ.txt
264a885b-95a5-456a-a821-140e65f6c008.FPKM-UQ.txt
dc3875c2-1a88-491c-bf98-fa344dd3b07e.FPKM-UQ.txt
f857575f-94b7-4415-af8e-8adecd9d166c.FPKM-UQ.txt
646f0e58-d861-4761-9fd5-2e48cd28753e.FPKM-UQ.txt
722becbc-9e08-4d7e-8dd6-415761cd84f3.FPKM-UQ.txt
95e82e2f-d87e-4785-9e8e-c6b41d8b73f0.FPKM-UQ.txt
501eb9e8-28ad-4329-934b-4279ccd65a19.FPKM-UQ.txt
f1bf5f82-1a74-4404-b

020fcc42-97ff-4959-b52f-f7f119c3f643.FPKM-UQ.txt
e8ebaff0-2f12-4834-bc45-57cf576fd45d.FPKM-UQ.txt
dde393cc-d889-4a29-ace8-46eb0919286a.FPKM-UQ.txt
b8592487-a20c-4d25-9131-51315d342a22.FPKM-UQ.txt
1be9778b-7fa2-4807-b6f4-2eebda72b2e3.FPKM-UQ.txt
bcdc68db-c874-4097-9c46-b06e331caaf5.FPKM-UQ.txt
db17e1af-915c-47f9-88dd-519b5303d3f0.FPKM-UQ.txt
60fd76fd-8522-497d-b631-e550988a1064.FPKM-UQ.txt
bdba7172-5280-41f6-8082-dbf8983c4089.FPKM-UQ.txt
e74a575b-5045-4727-b6c7-304e7afc77da.FPKM-UQ.txt
d550e6a0-c834-40c5-92f1-bf53ffdc06ec.FPKM-UQ.txt
57a77d9e-c513-475f-bd5d-896d28b19ac1.FPKM-UQ.txt
f8a3435f-aa69-49fe-9617-56e68858aecc.FPKM-UQ.txt
1ad88d50-b62a-4a36-9304-958944b751f6.FPKM-UQ.txt
2eb90caf-5d1f-48c0-b0c6-8d13c6ef7a4c.FPKM-UQ.txt
53cd4bb4-5c52-46b6-a37c-c442b536c5e4.FPKM-UQ.txt
f053f20c-775d-43aa-bfcf-cf816cc0dc2e.FPKM-UQ.txt
456ef025-b7f0-4040-90dd-5bcc2e476bfc.FPKM-UQ.txt
3235ff77-ae0d-49a7-9d17-48d12297a69c.FPKM-UQ.txt
026a42e7-aae9-4d96-9e5a-e6159f94db11.FPKM-UQ.txt
f1f609a2-4fd6-48fc-8